In [1]:
import figPlot
import numpy as np
import pandas as pd
from scipy import misc
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization
from keras.layers import LeakyReLU, MaxPooling2D, GlobalMaxPooling2D, Dense
import webbrowser
import os
import glob
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
#图片集存放路径 
img_path = '../MusicInputFig_Eng/*' 
imgs = glob.glob(img_path) #读取该路径下所有图片 返回路径列表list

In [3]:
def imread(f):
    x = misc.imread(f, mode='RGB')
    return x.astype(np.float32) / 255 * 2 - 1

#create AutoEncoder
def AutoEncoder(ImgShape):
    '''
    paragrams: ImgData size is [height, weight]
    '''
    img_height = ImgShape[0]
    img_weight = ImgShape[1]

    z_dim = 128 # 隐变量维度
    
    # 编码器（卷积与最大池化）
    x_in = Input(shape=(img_height, img_weight, 3))
    x = x_in
    
    for i in range(4):
        x = Conv2D(int(z_dim / 2**(3-i)), 
                   kernel_size=(3,3), padding='SAME')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)
        x = MaxPooling2D((2, 2))(x)

    for i in range(2):
        x = Conv2D(z_dim,
                   kernel_size=(3,3),
                   padding='SAME')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)
    
    x = GlobalMaxPooling2D()(x) # 全局特征
    
    z_mean = Dense(z_dim)(x) # 均值，也就是最终输出的编码
    encoder = Model(x_in, z_mean) # 总的编码器就是输出z_mean
    return encoder

def calcuSimMus(zs):
    topn = 11
    idxs = ((zs**2).sum(1) + (zs[1500]**2).sum() - 2 * np.dot(zs, zs[1500])).argsort()[1:topn]
    similar_list = idxs.tolist()
    return similar_list

def getMusicName(simiList):
    Music_list = []
    img_data = pd.read_csv('../Data/nameIndex.csv', names=['id', 'name'])
    for sims in simiList:
        simImage = img_data[img_data['id']==int(sims)]
        simName = simImage['name'].values[0]
        Music_list.append(simName)
        print(simName)
    return Music_list

def srcGet(path, name):
    return path + name

def nameGet(name):
    return os.path.splitext(name)[0]

def simPer(N):
    return [str(round((N-i) / (N/10)-9, 2)*100)+'%' for i in range(1, 11)]
percentage = simPer(1500)

In [4]:
#得到训练集  注意: 这步很耗内存
img_train = []
index = 0
for imgPath in tqdm(iter(imgs[:1500])):
    fname = os.path.splitext(imgPath.split('/')[-1])[0] #获得歌曲名
    index = index+1
    img = imread(imgPath)
    img_train.append(img)
    del img

0it [00:00, ?it/s]/home/bob/.conda/envs/pythonDL/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
1500it [00:40, 37.49it/s]


In [5]:
MusicName = 'DAOKO,米津玄師 - 打上花火.mp3'

In [6]:
MusicDir = '../testMusic/music/'
FigDir = '../testMusic/musicFig/'
absolutelyMusicDir = '/home/bob/Desktop/Music_Recommend/testMusic/music/'
musicDir = '/home/bob/Music/music_dataset/'

musicPath = MusicDir + MusicName
musicFigPath = FigDir + MusicName
FigPath = musicFigPath + '.png'

In [7]:
img = imread(FigPath)

/home/bob/.conda/envs/pythonDL/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


In [8]:
img_train.append(img)

In [9]:
image_train = np.array(img_train)  
x_train = image_train
del img_train

In [10]:
encoder = AutoEncoder(np.expand_dims(img, axis=0))

In [11]:
zs = encoder.predict(x_train, verbose=True)
print(zs.mean()) # 查看均值（简单观察先验分布有没有达到效果）
print(zs.std()) # 查看方差（简单观察先验分布有没有达到效果）

1501/1501 [==============================] - 24s 16ms/step
-0.0012588811
0.120184205


In [12]:
np.savetxt('/home/bob/Desktop/MusicVec.txt', zs, delimiter=',')

In [13]:
musicVec = np.loadtxt('/home/bob/Desktop/MusicVec.txt', delimiter=',')

In [14]:
simiList = calcuSimMus(musicVec)
simiList

[479, 1124, 273, 837, 525, 874, 1159, 53, 486, 924]

In [15]:
recMusic = getMusicName(simiList)

DAOKO (ダヲコ)-打上花火.m4a
AGA-Wonderful U.mp3
Wolves.mp3
Wonderful U_AGA.wav
Wonderful U (Demo Version).mp3
Libera-God Rest You Merry Gentlemen.m4a
Selena Gomez,Marshmello - Wolves.flac
Wolves_Selena Gomez.wav
Kory Burns - Count the Ways (Main Mix).mp3
Holly Brook-Saturdays.mp3


In [16]:
nameHtml = '../musicShow.html'
f = open(nameHtml, 'w')
message = """
<!DOCTYPE html>

<!DOCTYPE >
<html >

	<style type="text/css">
		body {
			color: #00ffff;
		}
		
		#player,
		#list {
			margin: 35px;
		}
		
		a,
		a:visited,
		a:active {
			color: #02F78E;
			text-decoration: none;
			font-size: 15px;
		}
		
		a:hover {
			color: #1abc9c;
			text-decoration: none;
		}
		
		#wrap {
            background-image: url(Data/timg2.jpeg);
			Background-size: cover;
			margin: 10px;
			padding: 10px 10px 10px 20px;
			width: 350px;
			-webkit-box-shadow: 0 0 15px #000;
			-moz-box-shadow: 0 0 15px #000;
			box-shadow: 0 0 15px #000;
			-webkit-backface-visibility: hidden;
			-webkit-border-radius: 2px;
			-moz-border-radius: 2px;
			border-radius: 2px;
		}
		
		#now,
		#mp3name {
			font-size:12px;
			font-family: 宋体;
			float: left;
			margin: 10px;
		}
		
		#title {
			color: #80FFFF;
			margin: 10px 10px 15px 10px;
			font-style: italic;
			font-weight: bold;
			font-size:23px;
			font-family: ;  //前三行字体
		}
		
		#search {
			margin : 10px;
		}
		
		#ftitle {
			margin-bottom: 10px;
			padding: 0px;
			font-size: 24px;
			font-weight: bold;
		}
		
		#stitle {
			margin: 0px;
			padding: 0px;
			font-size: 20px;
		}
		
	</style>

	<head>
		<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
		<title>HTML5音乐播放器</title>
		<meta name="keywords" content="HTML5音乐播放器。" />
	</head>

	<body > <!-- 背景图片-->
		<div id="wrap">
			<div id="list">
				<p id="ftitle">所选歌曲</p>
				<div id="title">
					<a href="javascript:void(0);" onclick="playmusic(0)">%s</a>
				</div>
				
				<p id="stitle">推荐歌曲</p><br />
				
				<form>
				
				<li>
					<a href="javascript:void(0);" onclick="playmusic(1)">%s</a>
				</li>
				<li>
					<a href="javascript:void(0);" onclick="playmusic(2)">%s</a>
				</li>
				<li>
					<a href="javascript:void(0);" onclick="playmusic(3)">%s</a>
				</li>
				<li>
					<a href="javascript:void(0);" onclick="playmusic(4)">%s</a>
				</li>
				<li>
					<a href="javascript:void(0);" onclick="playmusic(5)">%s</a>
				</li>
				<li>
					<a href="javascript:void(0);" onclick="playmusic(6)">%s</a>
				</li>
				<li>
					<a href="javascript:void(0);" onclick="playmusic(7)">%s</a>
				</li>
				<li>
					<a href="javascript:void(0);" onclick="playmusic(8)">%s</a>
				</li>
				<li>
					<a href="javascript:void(0);" onclick="playmusic(9)" >%s</a>
				</li>
                <li>
					<a href="javascript:void(0);" onclick="playmusic(10)" >%s</a>
				</li>
			</div>
			<audio id="player" controls="true">你的浏览器太烂了</audio>
			<div id="now">正在播放：</div><span id="mp3name">无</span>
			<div style="clear:both"></div>
		</div>
	</body>

	<script type="text/javascript">
		function playmusic(i) {
			var my = document.getElementById("player");
			switch(i) {
            	case 0:
					my.setAttribute("src", "%s");
					document.getElementById("mp3name").innerText = "正在获取歌曲。。。";
					my.addEventListener("canplaythrough",
						function() {
							document.getElementById("mp3name").innerText = "%s";
						}, false);
					my.play();
					break;
				case 1:
					my.setAttribute("src", "%s");
					document.getElementById("mp3name").innerText = "正在获取歌曲。。。";
					my.addEventListener("canplaythrough",
						function() {
							document.getElementById("mp3name").innerText = "%s";
						}, false);
					my.play();
					break;
				case 2:
					my.setAttribute("src", "%s");
					document.getElementById("mp3name").innerText = "正在获取歌曲。。。";
					my.addEventListener("canplaythrough",
						function() {
							document.getElementById("mp3name").innerText = "%s";
						}, false);
					my.play();
					break;
				case 3:
					my.setAttribute("src", "%s");
					document.getElementById("mp3name").innerText = "正在获取歌曲。。。";
					my.addEventListener("canplaythrough",
						function() {
							document.getElementById("mp3name").innerText = "%s";
						}, false);
					my.play();
					break;
				case 4:
					my.setAttribute("src", "%s");
					document.getElementById("mp3name").innerText = "正在获取歌曲。。。";
					my.addEventListener("canplaythrough",
						function() {
							document.getElementById("mp3name").innerText = "%s";
						}, false);
					my.play();
					break;
				case 5:
					my.setAttribute("src", "%s");
					document.getElementById("mp3name").innerText = "正在获取歌曲。。。";
					my.addEventListener("canplaythrough",
						function() {
							document.getElementById("mp3name").innerText = "%s";
						}, false);
					my.play();
					break;
				case 6:
					my.setAttribute("src", "%s");
					document.getElementById("mp3name").innerText = "正在获取歌曲。。。";
					my.addEventListener("canplaythrough",
						function() {
							document.getElementById("mp3name").innerText = "%s";
						}, false);
					my.play();
					break;
				case 7:
					my.setAttribute("src", "%s");
					document.getElementById("mp3name").innerText = "正在获取歌曲。。。";
					my.addEventListener("canplaythrough",
						function() {
							document.getElementById("mp3name").innerText = "%s";
						}, false);
					my.play();
					break;
				case 8:
					my.setAttribute("src", "%s");
					document.getElementById("mp3name").innerText = "正在获取歌曲。。。";
					my.addEventListener("canplaythrough",
						function() {
							document.getElementById("mp3name").innerText = "%s";
						}, false);
					my.play();
					break;
				case 9:
					my.setAttribute("src", "%s");
					document.getElementById("mp3name").innerText = "正在获取歌曲。。。";
					my.addEventListener("canplaythrough",
						function() {
							document.getElementById("mp3name").innerText = "%s";
						}, false);
					my.play();
					break;
                case 10:
					my.setAttribute("src", "%s");
					document.getElementById("mp3name").innerText = "正在获取歌曲。。。";
					my.addEventListener("canplaythrough",
						function() {
							document.getElementById("mp3name").innerText = "%s";
						}, false);
					my.play();
					break;
				default:
			}

		}
	</script>Design by Bob    
    """%(nameGet(MusicName), 
    nameGet(percentage[0]+' '+recMusic[0]), 
    nameGet(percentage[1]+' '+recMusic[1]), 
    nameGet(percentage[2]+' '+recMusic[2]), 
    nameGet(percentage[3]+' '+recMusic[3]), 
    nameGet(percentage[4]+' '+recMusic[4]), 
    nameGet(percentage[5]+' '+recMusic[5]), 
    nameGet(percentage[6]+' '+recMusic[6]), 
    nameGet(percentage[7]+' '+recMusic[7]), 
    nameGet(percentage[8]+' '+recMusic[8]), 
    nameGet(percentage[9]+' '+recMusic[9]), 
    srcGet(absolutelyMusicDir, MusicName),
    nameGet(MusicName),    
    srcGet(musicDir, recMusic[0]), 
    nameGet(recMusic[0]), 
    srcGet(musicDir, recMusic[1]), 
    nameGet(recMusic[1]), 
    srcGet(musicDir, recMusic[2]), 
    nameGet(recMusic[2]), 
    srcGet(musicDir, recMusic[3]), 
    nameGet(recMusic[3]), 
    srcGet(musicDir, recMusic[4]), 
    nameGet(recMusic[4]), 
    srcGet(musicDir, recMusic[5]), 
    nameGet(recMusic[5]), 
    srcGet(musicDir, recMusic[6]), 
    nameGet(recMusic[6]), 
    srcGet(musicDir, recMusic[7]), 
    nameGet(recMusic[7]), 
    srcGet(musicDir, recMusic[8]),
    nameGet(recMusic[8]),
    srcGet(musicDir, recMusic[9]),
    nameGet(recMusic[9])
    )


f.write(message)
f.close()

webbrowser.open(nameHtml, new=1)

True